In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install transformers
!pip install evaluate
!pip install accelerate -U

In [3]:
import sys
import torch
import pandas as pd
from transformers import BertTokenizer, BertModel, BertConfig
from torch.utils.data import DataLoader
kaggle = False
if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    if os.environ['KAGGLE_KERNEL_RUN_TYPE'] == 'Interactive':
        kaggle = True
        
if kaggle:
    sys.path.append('/kaggle/input/text-detoxification/Text-Detoxification')

    
#disabling wandb.ai
os.environ["WANDB_DISABLED"] = "true"


In [4]:
from src.data.make_dataset import ToxicTextClassificationDataset
from src.models.train_model import TrainToxicTextClassifier
from src.models.train_model import BERTClass
from src.models.train_model import ValidateToxicTextClassifier

In [5]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 64
TRAIN_BATCH_SIZE = 128
VALID_BATCH_SIZE = 64
EPOCHS = 3
LEARNING_RATE = 1e-04

In [6]:
file_path = 'data/raw/filtered.tsv'
if kaggle:
    file_path = '/kaggle/input/text-detoxification/Text-Detoxification/' + file_path


df = pd.read_csv(file_path, sep='\t')

checkpoint = 'bert-base-uncased'
model = BERTClass(checkpoint)
tokenizer = BertTokenizer.from_pretrained(checkpoint)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [7]:
from sklearn.model_selection import train_test_split


# Split the dataset into train, test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Create datasets using the split data
train_dataset = ToxicTextClassificationDataset(train_df.reset_index(drop=True), tokenizer, 64)
test_dataset = ToxicTextClassificationDataset(test_df.reset_index(drop=True), tokenizer, 64)


In [8]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(train_dataset, **train_params)
testing_loader = DataLoader(test_dataset, **test_params)

In [9]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs.squeeze(1), targets)

In [10]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [11]:
#ret = TrainToxicTextClassifier(model, training_loader, testing_loader, device, loss_fn, optimizer, EPOCHS, save_name = 'bert-base-uncased-finetuned')

In [12]:
#torch.save(model.state_dict(), 'ToxicClassificationDataset.pth')
model.load_state_dict(torch.load('models/ToxicClassificationDataset.pth'))

<All keys matched successfully>

In [13]:
ValidateToxicTextClassifier(model, testing_loader, device, best_f1 = 0)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\Yazan\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
C:\Users\Yazan\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2614: FutureW

{'accuracy': 0.8648897058823529, 'f1_score_macro': 0.8648886786316194}